### Librerías

In [1]:
import pandas as pd
import numpy as np
import bcchapi
from datetime import timedelta
from datetime import datetime
import re
import xlrd
import openpyxl
import os
import pyodbc

### API BDE

In [2]:
siete = bcchapi.Siete("pbustamante@vectorcapital.cl", "Vector.2025")
fecha_hoy = datetime.today().strftime('%Y-%m-%d')

## EURO
df_euro_obs = siete.cuadro(
series=["F072.CLP.EUR.N.O.D"],
nombres = ["euro"],
desde="2025-01-01",
hasta=fecha_hoy)

df_euro_obs = df_euro_obs.reset_index()
df_euro_obs.columns = ['Fecha', 'Euro TC']
df_euro_obs["Fecha"] = pd.to_datetime(df_euro_obs["Fecha"]).dt.strftime("%d-%m-%Y")

### USD
df_dolar_obs = siete.cuadro(
series=["F073.TCO.PRE.Z.D"],
nombres = ["dolar"],
desde="2025-01-01",
hasta=fecha_hoy)
df_dolar_obs = df_dolar_obs.reset_index()
df_dolar_obs.columns = ['Fecha', 'Dolar TC']
df_dolar_obs["Fecha"] = pd.to_datetime(df_dolar_obs["Fecha"]).dt.strftime("%d-%m-%Y")

### FBICE

In [3]:
def extraer_año_desde_encabezado_BICE(texto_columna):
    """Extrae el año desde el texto del encabezado (antes del símbolo '|')"""
    before_pipe = re.split(r'\|', texto_columna)[0].strip()
    match = re.search(r'\b(\d{4})\b', before_pipe)
    return match.group(1) if match else ""

def cargar_cartola_bice(ruta_archivo, tipo_mon='clp', df_euro_obs=None):
    df = pd.read_excel(ruta_archivo)
    text = df.columns[0]
    year = extraer_año_desde_encabezado_BICE(text)
    
    # CLEAN ENTRADA y SALIDA
    df = df.iloc[9:]
    df.columns = df.iloc[0]
    df = df.iloc[2:].reset_index(drop=True)
    first_empty_index = df[df.isnull().all(axis=1)].index.min() # SALIDA
    if pd.notna(first_empty_index):
        df = df.iloc[:first_empty_index]
    # Fix fecha (completar y agregar año) y formatear dd-mm-yyyy; sacar col saldo contable
    df["Fecha"] = df["Fecha"].replace("-", pd.NA).ffill()
    df["Fecha"] = df["Fecha"].astype(str).str.strip() + "-" + year
    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d-%m-%Y", errors="coerce")
    df["Fecha"] = df["Fecha"].dt.strftime("%d-%m-%Y")
    if 'Saldo Contable' in df.columns:
        del df['Saldo Contable']
    # Si tipo_mon = 'eur', unir con tipo de cambio y convertir
    if tipo_mon.lower() == 'eur':
        if df_euro_obs is None:
            raise ValueError("Debe proporcionar df_euro_obs cuando tipo_mon es 'eur'")

        df = pd.merge(df, df_euro_obs, how='left', on='Fecha')

        # a numéricos y nuevos campos
        df['Cargos'] = pd.to_numeric(df['Cargos'], errors='coerce')
        df['Abonos'] = pd.to_numeric(df['Abonos'], errors='coerce')
        df['Euro TC'] = pd.to_numeric(df['Euro TC'], errors='coerce')
        df['Cargos Moneda de Origen'] = df['Cargos'] / df['Euro TC']
        df['Abono Moneda de Origen'] = df['Abonos'] / df['Euro TC']

    return df

### CARTOLAS

In [4]:
ruta_dir = r"Y:\Cartolas bancarias Contabilidad\2025\Mayo 2025\BICE\BICE 01-35659-3"
archivos = os.listdir(ruta_dir)
archivos_xlsx = [f for f in archivos if f.lower().endswith('.xlsx') and os.path.isfile(os.path.join(ruta_dir, f))]

dfs = []
for archivo in archivos_xlsx:
    ruta_archivo = os.path.join(ruta_dir, archivo)
    df = cargar_cartola_bice(ruta_archivo, tipo_mon='clp')
    dfs.append(df)

cartola_clp_RACIONAL = pd.concat(dfs, ignore_index=True)

In [5]:
df_to_rut = cartola_clp_RACIONAL

# General
def extraer_rut(texto, n=1):
    """
    Extrae el n-ésimo RUT chileno del texto (por defecto el primero).
    Si no encuentra, retorna None.
    """
    # Busca todos los RUTs con 7 u 8 dígitos, guion y dígito/K
    ruts = re.findall(r'(\d{7,8}-[0-9Kk])', texto.replace('.', ''))
    if len(ruts) >= n:
        return ruts[n-1].upper()
    return None

# Ejemplo: para extraer el primer RUT
df_to_rut['IDENTIFICADOR'] = df_to_rut['Descripción'].apply(lambda x: extraer_rut(str(x), n=1))

# Extrae el RUT desde cualquier parte de la descripción
df_to_rut['IDENTIFICADOR'] = df_to_rut['Descripción'].apply(lambda x: extraer_rut(str(x)))

# Excepción
cond_excepcional = df_to_rut['Descripción'].str.contains(r'^Transf\. via Internet a cuenta .*desde VECTOR CAPITAL CORREDORES DE BOLSA SPA', regex=True)
df_to_rut.loc[cond_excepcional, 'IDENTIFICADOR'] = '076513680-6'

# Excepción
cond_excepcional = df_to_rut['Descripción'].str.contains(r'^Transf\. via Internet a cuenta .*desde VECTOR CAPITAL CORREDORES DE BOLSA SPA', regex=True)
df_to_rut.loc[cond_excepcional, 'IDENTIFICADOR'] = '076513680-6'
df_to_rut['IDENTIFICADOR'] = df_to_rut['IDENTIFICADOR'].str.replace('.', '', regex=False)

## Extraer hora depósito para el match con dashboard RC
def extraer_fecha_hora(texto):
    match = re.search(r'(\d{2}/\d{2}/\d{4})\s*a\s*las\s*(\d{2}:\d{2})', str(texto))
    if match:
        return f"{match.group(1)} {match.group(2)}"
    return None

df_to_rut['Fecha_transferencia'] = df_to_rut['Descripción'].apply(extraer_fecha_hora)
df_to_rut['Fecha_transferencia'] = pd.to_datetime(df_to_rut['Fecha_transferencia'], format='%d/%m/%Y %H:%M')
df_to_rut['Fecha_transferencia'] = df_to_rut['Fecha_transferencia'].dt.strftime('%d-%m-%Y %H:%M')

Debug

In [6]:
df_to_rut[df_to_rut['Descripción'] == 'Transferencia de CLAUDIO ALBERTO CAMPOS HAN Rut 19222315-6 desde Banco BICE a VECTOR CAPITAL CORREDORES DE BOLSA SPA Rut 76513680-6 a Cuenta Corriente de Banco BICE,el 19/05/2025 a las 07:19 hrs.']

9,Fecha,Documento,Descripción,Cargos,Abonos,IDENTIFICADOR,Fecha_transferencia
18273,19-05-2025,NaN,Transferencia de CLAUDIO ALBERTO CAMPOS HAN Ru...,0,35000,19222315-6,19-05-2025 07:19


### QRY

In [7]:
F_i  = '2025-04-29'
F    = '2025-06-02' 

query_universal = {

        'query_APORET': f"""
        SELECT 
               CARGO_ABONO, 
               NUM_CUENTA, 
               IDENTIFICADOR,
               NOMBRE_CLI, 
               COD_MOV, 
               DSC_MOV_CAJA, 
               FECHA_MOVIMIENTO,
               FECHA_LIQUIDACION, 
               MONTO, 
               NOMBRE_ASESOR, 
               TIPO_CAJA,
               MOV_AUTOMATICO,
               OBS_MOV_CAJA

        FROM [Capitaria].[dbo].[MOV_CAJA_CLI_JGG]
        WHERE COD_MOV IN (
            'APO_PAT',
            'APO_PAT_AT',
            'APO_PAT_BT',
            'APO_PAT_LI',
            'APO_PAT_RC'
            )
        AND FECHA_MOVIMIENTO BETWEEN CONVERT(datetime, '{F_i}', 120) AND CONVERT(datetime, '{F}', 120)
        """
}

conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
df_APORET_detalle = pd.read_sql(query_universal['query_APORET'], conn)
conn.close()

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_15732\789851610.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_APORET_detalle = pd.read_sql(query_universal['query_APORET'], conn)


In [16]:
## FILTRAR CARTOLA
# Filtrar solo depósitos (Abonos > 0) y que IDENTIFICADOR no sea vacío ni nulo
df_to_rut = df_to_rut[(df_to_rut['Abonos'] > 0) & (df_to_rut['IDENTIFICADOR'].notna()) & (df_to_rut['IDENTIFICADOR'].str.strip() != '')]

# Asegura que las fechas sean datetime
df_to_rut['Fecha'] = pd.to_datetime(df_to_rut['Fecha'], format='%d-%m-%Y', errors='coerce')
df_APORET_detalle['FECHA_MOVIMIENTO'] = pd.to_datetime(df_APORET_detalle['FECHA_MOVIMIENTO'], errors='coerce')

# Ordena ambos DataFrames por IDENTIFICADOR, MONTO/Abonos y fecha
df_to_rut = df_to_rut.sort_values(['IDENTIFICADOR', 'Abonos', 'Fecha'])
df_APORET_detalle = df_APORET_detalle.sort_values(['IDENTIFICADOR', 'MONTO', 'FECHA_MOVIMIENTO'])

# Normaliza los montos a float y luego a int para evitar diferencias por decimales
df_to_rut['Abonos'] = pd.to_numeric(df_to_rut['Abonos'], errors='coerce').fillna(0).astype(int)
df_APORET_detalle['MONTO'] = pd.to_numeric(df_APORET_detalle['MONTO'], errors='coerce').fillna(0).astype(int)

# ---- Rut k to K
df_to_rut['IDENTIFICADOR'] = df_to_rut['IDENTIFICADOR'].str.upper()
df_APORET_detalle['IDENTIFICADOR'] = df_APORET_detalle['IDENTIFICADOR'].str.upper()

# Ahora crea la clave
df_to_rut['clave'] = df_to_rut['IDENTIFICADOR'].astype(str) + '_' + df_to_rut['Abonos'].astype(str)
df_APORET_detalle['clave'] = df_APORET_detalle['IDENTIFICADOR'].astype(str) + '_' + df_APORET_detalle['MONTO'].astype(str)

# mod
df_to_rut = df_to_rut.reset_index().rename(columns={'index': 'idx_cartola'})


# Para cada fila de df_to_rut, busca el match más cercano en fecha en df_APORET_detalle
resultados = []
usados = set()

for idx, row in df_to_rut.iterrows():
    candidatos = df_APORET_detalle[df_APORET_detalle['clave'] == row['clave']]
    if not candidatos.empty:
        # Calcula la diferencia absoluta de días
        candidatos = candidatos.assign(diff_days=(candidatos['FECHA_MOVIMIENTO'] - row['Fecha']).abs())
        # Elige el más cercano que no haya sido usado
        candidatos = candidatos[~candidatos.index.isin(usados)]
        if not candidatos.empty:
            elegido = candidatos.loc[candidatos['diff_days'].idxmin()]
            usados.add(elegido.name)
            merged_row = {**row, **elegido}
            resultados.append(merged_row)
        else:
            # No hay candidatos disponibles (ya usados)
            merged_row = {**row}
            for col in df_APORET_detalle.columns:
                if col not in merged_row:
                    merged_row[col] = np.nan
            resultados.append(merged_row)
    else:
        # No hay candidatos
        merged_row = {**row}
        for col in df_APORET_detalle.columns:
            if col not in merged_row:
                merged_row[col] = np.nan
        resultados.append(merged_row)

df_match = pd.DataFrame(resultados)

# ---- Solo dejar movimientos realmente matcheados (ambos lados tienen datos)
df_match = df_match[df_match['MONTO'].notna()]
# No match en cartola: los que no lograron match en GPI
no_match_cartola = df_to_rut[~df_to_rut['idx_cartola'].isin(df_match['idx_cartola'])]

# No match en APORET (los que no fueron usados)
no_match_aporet = df_APORET_detalle[~df_APORET_detalle.index.isin(usados)]

# Exporta a Excel
with pd.ExcelWriter('Temporal/ViernesCartolaGPI_realmatch3.xlsx') as writer:
    df_match.to_excel(writer, sheet_name='Match', index=False)
    df_to_rut.to_excel(writer, sheet_name='Cartola', index=False)
    df_APORET_detalle.to_excel(writer, sheet_name='GPI', index=False)
    no_match_cartola.to_excel(writer, sheet_name='No_Match_CARTOLA', index=False)
    no_match_aporet.to_excel(writer, sheet_name='No_Match_APORET', index=False)

In [15]:
df_match.shape

(31158, 21)

In [9]:
aidi =  '8234253-7'
display(df_to_rut[df_to_rut['IDENTIFICADOR'] == aidi])
display(df_APORET_detalle[df_APORET_detalle['IDENTIFICADOR'] == aidi])


9,Fecha,Documento,Descripción,Cargos,Abonos,IDENTIFICADOR,Fecha_transferencia,clave
13248,2025-05-12,22706411,Abono por transferencia de Hernan Raul Contrer...,0,2000000,8234253-7,10-05-2025 09:01,8234253-7_2000000


,CARGO_ABONO,NUM_CUENTA,IDENTIFICADOR,NOMBRE_CLI,COD_MOV,DSC_MOV_CAJA,FECHA_MOVIMIENTO,FECHA_LIQUIDACION,MONTO,NOMBRE_ASESOR,TIPO_CAJA,MOV_AUTOMATICO,OBS_MOV_CAJA,clave


### Dashboard

In [ ]:
"""Dashboard_Mayo = pd.read_csv("Input/Dashboard_Mayo.csv", sep=',', engine='python')

def format_rut_column(df, col='rut'):
    df[col] = df[col].astype(str).str.replace(r'(\d+)(\d{1})$', r'\1-\2', regex=True)
    return df
Dashboard_Mayo = format_rut_column(Dashboard_Mayo, col='rut')

## Formatear 
Dashboard_Mayo['Cargos'] = np.where(Dashboard_Mayo['transactionType'] == 'withdrawal', Dashboard_Mayo['amountCLP'], 0)
Dashboard_Mayo['Abonos'] = np.where(Dashboard_Mayo['transactionType'] == 'deposit', Dashboard_Mayo['amountCLP'], 0)

del Dashboard_Mayo['amountCLP']
del Dashboard_Mayo['id']
del Dashboard_Mayo['transactionType']

Dashboard_Mayo = Dashboard_Mayo.rename(columns={
    'rut': 'IDENTIFICADOR',
    'createdAtLocal': 'Fecha'})


Dashboard_Mayo['Fecha_normalizada'] = pd.to_datetime(Dashboard_Mayo['Fecha'], errors='coerce')
Dashboard_Mayo['Fecha_normalizada'] = Dashboard_Mayo['Fecha_normalizada'].dt.strftime('%d-%m-%Y %H:%M')
Dashboard_Mayo['Cargos'] = pd.to_numeric(Dashboard_Mayo['Cargos'], errors='coerce')
Dashboard_Mayo['Abonos'] = pd.to_numeric(Dashboard_Mayo['Abonos'], errors='coerce')

def formatear_identificador(df, col='IDENTIFICADOR'):
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r'[^0-9kK]', '', regex=True)  # Elimina caracteres no válidos
        .str.upper()  # Convierte la K a mayúscula
        .str.replace(r'(\d+)([0-9K])$', r'\1-\2', regex=True)  # Agrega el guion antes del último dígito/letra
    )
    return df

Dashboard_Mayo = formatear_identificador(Dashboard_Mayo, col='IDENTIFICADOR')"""

### BiceWebhook

In [ ]:
"""BiceWebhook = pd.read_excel("bice.xlsx")"""

### test

In [ ]:
"""cartola_clp1 = cargar_cartola_bice("Y:\PROYECTO CONCILIACION\EJEMPLOS POWER QUARY\BICE\BICE 01-29316-8/30-04.xlsx", tipo_mon='clp')
cartola_clp_RACIONAL = cargar_cartola_bice("Y:\PROYECTO CONCILIACION\EJEMPLOS POWER QUARY\BICE\BICE 01-35659-3/30-04.xlsx", tipo_mon='clp') 
cartola_clp_BETTERPLAN = cargar_cartola_bice("Y:\PROYECTO CONCILIACION\EJEMPLOS POWER QUARY\BICE\BICE 01-36253-4/30-04.xlsx", tipo_mon='clp') 
cartola_clp_ZESTY = cargar_cartola_bice("Y:\PROYECTO CONCILIACION\EJEMPLOS POWER QUARY\BICE\BICE 01-37487-7/30-04.xlsx", tipo_mon='clp') 
cartola_clp5 = cargar_cartola_bice("Y:\PROYECTO CONCILIACION\EJEMPLOS POWER QUARY\BICE\BICE 01-38571-2/30-04.xlsx", tipo_mon='clp') ## SHINKANSEN
cartola_clp6 = cargar_cartola_bice("Y:\PROYECTO CONCILIACION\EJEMPLOS POWER QUARY\BICE\BICE 01-40540-3/07-03 AL 07-04.xlsx", tipo_mon='clp')
cartola_eur = cargar_cartola_bice("Y:\PROYECTO CONCILIACION\EJEMPLOS POWER QUARY\BICE\BICE 14-20-101366-7 EURO/02-01 AL 02-04.xlsx", tipo_mon='eur', df_euro_obs=df_euro_obs)"""

In [ ]:

# query_universal = {  
#     'query_APORET': f"""  
#         SELECT  
#             CARGO_ABONO,  
#             NUM_CUENTA,  
#             IDENTIFICADOR,  
#             NOMBRE_CLI,  
#             COD_MOV,  
#             DSC_MOV_CAJA,  
#             FECHA_MOVIMIENTO,  
#             FECHA_LIQUIDACION,  
#             MONTO,  
#             NOMBRE_ASESOR,  
#             TIPO_CAJA,  
#             MOV_AUTOMATICO,  
#             OBS_MOV_CAJA  
#         FROM [Capitaria].[dbo].[MOV_CAJA_CLI_JGG]  
#         WHERE IDENTIFICADOR = '17665491-0'  and
#         FECHA_MOVIMIENTO BETWEEN CONVERT(datetime, '{F_i}', 120) AND CONVERT(datetime, '{F}', 120)
#     """  
# }

# conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# df_APORET_validacion = pd.read_sql(query_universal['query_APORET'], conn)
# conn.close()

# df_APORET_validacion.sort_values(by='FECHA_MOVIMIENTO', ascending=False).to_excel("Temporal/Cliente_validacion1.xlsx", index=False)
# df_APORET_validacion
# #and DSC_MOV_CAJA = 'APORTE PATRIMONIAL RC'

In [ ]:
"""## webhook aparece la fecha del depósito y la fecha donde va a aparecer en cartola

validar = '17665491-0'

print("------BICE")
display(cartola_clp_RACIONAL[cartola_clp_RACIONAL['IDENTIFICADOR'] == validar])
display(BiceWebhook[BiceWebhook['IDENTIFICADOR'] == validar])

print("------Dashboard")
display(Dashboard_Mayo[Dashboard_Mayo['IDENTIFICADOR'] == validar])
print("------GPI")
display(df_APORET_detalle[df_APORET_detalle['IDENTIFICADOR'] == validar])"""

In [ ]:
"""import pandas as pd

# Asegura que las fechas sean del tipo correcto
cartola_clp_RACIONAL['Fecha'] = pd.to_datetime(cartola_clp_RACIONAL['Fecha'], format='%d-%m-%Y')
df_APORET_detalle['FECHA_MOVIMIENTO'] = pd.to_datetime(df_APORET_detalle['FECHA_MOVIMIENTO'])

# Merge por IDENTIFICADOR y monto (Abonos <-> MONTO)
df_merge = pd.merge(
    cartola_clp_RACIONAL,
    df_APORET_detalle,
    left_on=['IDENTIFICADOR', 'Abonos'],
    right_on=['IDENTIFICADOR', 'MONTO'],
    suffixes=('_CARTOLA', '_APORET'),
    how='left'
)

# Calcula la diferencia de días
df_merge['DIF_DIAS'] = (df_merge['FECHA_MOVIMIENTO'] - df_merge['Fecha']).dt.days

# Filtra matches: FECHA_MOVIMIENTO es igual o hasta 5 días mayor que Fecha
df_match = df_merge[(df_merge['DIF_DIAS'] >= 0) & (df_merge['DIF_DIAS'] <= 5)].copy()

# Para obtener los no match de cartola_clp_RACIONAL:
matched_cartola_idx = df_match['Fecha'].astype(str) + df_match['IDENTIFICADOR'].astype(str) + df_match['Abonos'].astype(str)
cartola_idx = cartola_clp_RACIONAL['Fecha'].astype(str) + cartola_clp_RACIONAL['IDENTIFICADOR'].astype(str) + cartola_clp_RACIONAL['Abonos'].astype(str)
no_match_cartola = cartola_clp_RACIONAL[~cartola_idx.isin(matched_cartola_idx)]

# Para obtener los no match de df_APORET_detalle:
matched_aporet_idx = df_match['FECHA_MOVIMIENTO'].astype(str) + df_match['IDENTIFICADOR'].astype(str) + df_match['MONTO'].astype(str)
aporet_idx = df_APORET_detalle['FECHA_MOVIMIENTO'].astype(str) + df_APORET_detalle['IDENTIFICADOR'].astype(str) + df_APORET_detalle['MONTO'].astype(str)
no_match_aporet = df_APORET_detalle[~aporet_idx.isin(matched_aporet_idx)]

# Exporta a Excel con varias pestañas
with pd.ExcelWriter('Temporal/output_cartola_match_5dias.xlsx') as writer:
    df_match.to_excel(writer, sheet_name='Match', index=False)
    no_match_cartola.to_excel(writer, sheet_name='No_Match_CARTOLA', index=False)
    no_match_aporet.to_excel(writer, sheet_name='No_Match_APORET', index=False)"""

In [ ]:

# query_universal = {

#         'query_APORET': f"""
#         SELECT 
#                CARGO_ABONO, 
#                NUM_CUENTA, 
#                IDENTIFICADOR,

               
#                NOMBRE_CLI, 
#                COD_MOV, 
#                DSC_MOV_CAJA, 
#                FECHA_MOVIMIENTO,
#                FECHA_LIQUIDACION, 
#                MONTO, 
#                NOMBRE_ASESOR, 
#                TIPO_CAJA,
#                MOV_AUTOMATICO,
#                OBS_MOV_CAJA

#         FROM [Capitaria].[dbo].[Movimiento_caja_liquidado]
#         where IDENTIFICADOR = '{validar}'
         
#         """
# }

# conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# df_APORET_validacion = pd.read_sql(query_universal['query_APORET'], conn)
# conn.close()

# df_APORET_validacion.to_excel("validar.xlsx")

# #df_APORET_validacion[df_APORET_validacion['COD_MOV'] == 'APO_PAT']

### Match Cartola Dashboard

In [ ]:
"""# Ordena ambos DataFrames
df_to_rut = df_to_rut.sort_values(['IDENTIFICADOR', 'Fecha_transferencia', 'Abonos', 'Cargos'])
Dashboard_Mayo = Dashboard_Mayo.sort_values(['IDENTIFICADOR', 'Fecha_normalizada', 'Abonos', 'Cargos'])


        def fifo_match_cartola_dashboard(df_to_rut, Dashboard_Mayo):
    df_dash = Dashboard_Mayo.copy()
    df_dash['usado'] = False
    merged_rows = []
    for idx, row in df_to_rut.iterrows():
        mask = (
            (df_dash['IDENTIFICADOR'] == row['IDENTIFICADOR']) &
            (df_dash['Abonos'] == row['Abonos']) &
            (df_dash['Cargos'] == row['Cargos']) &
            (df_dash['Fecha_normalizada'] == row['Fecha_transferencia']) &
            (~df_dash['usado'])
        )
        match = df_dash[mask]
        if not match.empty:
            first_match_idx = match.index[0]
            dash_row = df_dash.loc[first_match_idx].copy()
            dash_row = dash_row.add_suffix('_DASH')
            merged_row = {**row, **dash_row.to_dict()}
            merged_rows.append(merged_row)
            df_dash.at[first_match_idx, 'usado'] = True
        else:
            dash_cols = [col for col in df_dash.columns]
            merged_row = {**row}
            for col in dash_cols:
                merged_row[col + '_DASH'] = None
            merged_rows.append(merged_row)
    # Agrega los que quedaron sin usar en Dashboard_Mayo
    no_match_dash = df_dash[~df_dash['usado']]
    for idx, row in no_match_dash.iterrows():
        empty_cols = {col: None for col in df_to_rut.columns}
        dash_row = row.add_suffix('_DASH').to_dict()
        merged_row = {**empty_cols, **dash_row}
        merged_rows.append(merged_row)
    return pd.DataFrame(merged_rows)

df_result = fifo_match_cartola_dashboard(df_to_rut, Dashboard_Mayo)

no_match_cartola = df_result[df_result['Abonos_DASH'].isna()]
no_match_dashboard = df_result[df_result['Abonos'].isna()]
no_match_total = pd.concat([no_match_cartola, no_match_dashboard]).drop_duplicates()

with pd.ExcelWriter("Output/Dashboard_Cartola_MERGE2.xlsx") as writer:
    df_to_rut.to_excel(writer, sheet_name="Rut desde cartola", index=False)
    Dashboard_Mayo.to_excel(writer, sheet_name="Dashboard", index=False)
    df_result.to_excel(writer, sheet_name="Conciliado", index=False)
    no_match_cartola.to_excel(writer, sheet_name="No_match_cartola", index=False)
    no_match_dashboard.to_excel(writer, sheet_name="No_match_dashboard", index=False)
    no_match_total.to_excel(writer, sheet_name="No_match_total", index=False)"""

#### Conciliar BD con Cartolas

In [ ]:
"""df_APORET_detalle['IDENTIFICADOR'] = df_APORET_detalle['IDENTIFICADOR'].str.replace('k', 'K', regex=False)
df_to_rut['IDENTIFICADOR'] = df_to_rut['IDENTIFICADOR'].str.replace('.', '', regex=False)


# ORDENAR AMBOS DF PARA APLICAR FIFO
df_to_rut['Fecha'] = pd.to_datetime(df_to_rut['Fecha'], format='%d-%m-%Y', errors='coerce')
df_APORET_detalle['FECHA_MOVIMIENTO'] = pd.to_datetime(df_APORET_detalle['FECHA_MOVIMIENTO'], errors='coerce')
df_to_rut = df_to_rut.sort_values(['IDENTIFICADOR', 'Abonos', 'Fecha'])
df_APORET_detalle = df_APORET_detalle.sort_values(['IDENTIFICADOR', 'MONTO', 'FECHA_MOVIMIENTO'])

# Parapara cada fila de df_to_rut, busca el primer match disponible en df_APORET_detalle
def fifo_merge(df_cartola, df_aporet):
    df_cartola = df_cartola.copy()
    df_aporet = df_aporet.copy()
    df_aporet['usado'] = False
    merged_rows = []
    for idx, row in df_cartola.iterrows():
        mask = (
            (df_aporet['IDENTIFICADOR'] == row['IDENTIFICADOR']) &
            (df_aporet['MONTO'] == row['Abonos']) &
            (~df_aporet['usado'])
        )
        match = df_aporet[mask]
        if not match.empty:
            first_match_idx = match.index[0]
            merged_row = {**row, **df_aporet.loc[first_match_idx]}
            merged_rows.append(merged_row)
            df_aporet.at[first_match_idx, 'usado'] = True
        else:
            merged_rows.append({**row, **{col: None for col in df_aporet.columns}})
    return pd.DataFrame(merged_rows)

df_Conciliado = fifo_merge(df_to_rut, df_APORET_detalle)

no_match = df_Conciliado[df_Conciliado['MONTO'].isna()]
with pd.ExcelWriter("Output/Conciliacion_Week2.xlsx") as writer:
    df_to_rut.to_excel(writer, sheet_name="Rut desde cartola", index=False)
    df_APORET_detalle.to_excel(writer, sheet_name="APORET_detalle", index=False)
    df_Conciliado.to_excel(writer, sheet_name="Conciliado_todos", index=False)
    no_match.to_excel(writer, sheet_name="No_match", index=False)"""

### Validación de la conciliación

In [ ]:
# F_i = '2025-05-01'
# F = '2025-05-30'

# query_universal = {  
#     'query_APORET': f"""  
#         SELECT  
#             CARGO_ABONO,  
#             NUM_CUENTA,  
#             IDENTIFICADOR,  
#             NOMBRE_CLI,  
#             COD_MOV,  
#             DSC_MOV_CAJA,  
#             FECHA_MOVIMIENTO,  
#             FECHA_LIQUIDACION,  
#             MONTO,  
#             NOMBRE_ASESOR,  
#             TIPO_CAJA,  
#             MOV_AUTOMATICO,  
#             OBS_MOV_CAJA  
#         FROM [Capitaria].[dbo].[MOV_CAJA_CLI_JGG]  
#         WHERE IDENTIFICADOR = '18117065-4'  and
#         FECHA_MOVIMIENTO BETWEEN CONVERT(datetime, '{F_i}', 120) AND CONVERT(datetime, '{F}', 120)
#     """  
# }

# conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# df_APORET_validacion = pd.read_sql(query_universal['query_APORET'], conn)
# conn.close()

# df_APORET_validacion.sort_values(by='FECHA_MOVIMIENTO', ascending=False).to_excel("Temporal/Cliente_validacion1.xlsx", index=False)

# #and DSC_MOV_CAJA = 'APORTE PATRIMONIAL RC'

Validación

In [ ]:

# query_universal = {

#         'query_APORET': f"""
#         SELECT 
#                CARGO_ABONO, 
#                NUM_CUENTA, 
#                IDENTIFICADOR,
#                NOMBRE_CLI, 
#                COD_MOV, 
#                DSC_MOV_CAJA, 
#                FECHA_MOVIMIENTO,
#                FECHA_LIQUIDACION, 
#                MONTO, 
#                NOMBRE_ASESOR, 
#                TIPO_CAJA,
#                MOV_AUTOMATICO,
#                OBS_MOV_CAJA

#         FROM [Capitaria].[dbo].[Movimiento_caja_liquidado]
#         where IDENTIFICADOR = '20341629-6'
#         """
# }

# conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# df_APORET_validacion = pd.read_sql(query_universal['query_APORET'], conn)
# conn.close()

# df_APORET_validacion.sort_values(by='FECHA_MOVIMIENTO', ascending=False).head(20)

### Carta Gantt

In [ ]:
import plotly.figure_factory as ff

df = [dict(Task="Estandarizar Cartolas Bancarias", Start='2025-06-02', Finish='2025-06-08', Resource='Completo'),
      dict(Task="Aplicar conciliación BBDD, Cartolas, Dashboard", Start='2025-06-04', Finish='2025-06-22', Resource='Incompleto'),
      dict(Task="Conciliación Ejecutivos", Start='2025-06-16', Finish='2025-06-22', Resource='Sin partir'),
      dict(Task="Visualización mediante correo/reporte", Start='2025-06-23', Finish='2025-06-29', Resource='Sin partir')]

colors = {
    'Sin partir': 'rgb(220, 0, 0)',
    'Incompleto': (1, 0.9, 0.16),
    'Completo': 'rgb(0, 255, 100)'
}

fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True, group_tasks=True)

# Aumentar tamaño de fuente
fig.update_layout(
    font=dict(
        size=16  # Cambia este número para ajustar el tamaño
    )
)

fig.show()


In [ ]:
colors = {
    'Estandarizar Cartolas Bancarias': 'rgb(0, 102, 204)',
    'Cruce BBDD y Cartolas': 'rgb(0, 102, 204)',
    'Conciliación de cartolas bancarias con GPI y Dashboard de Racional': 'rgb(0, 102, 204)',
    'Integrar todas las plataformas en la conciliación de cartolas bancarias BICE': 'rgb(102, 204, 255)',
    'Comienzo de estandarización y conciliación con banco Security con Plataforma Racional': 'rgb(0, 204, 150)',
    'Conciliación con todas las plataformas en banco Security': 'rgb(204, 204, 0)',
    'Entrega de reportes. Integración de código Python con visualización en Power BI': 'rgb(255, 153, 51)',
    'Integración con Power BI. Entrega de reportes. Toma de decisiones en base a resultados': 'rgb(255, 102, 102)'
}

In [ ]:
import plotly.figure_factory as ff

df = [
    dict(Task="Semana 02 – 06 Junio", Start='2025-06-02', Finish='2025-06-08', Resource='Estandarizar Cartolas Bancarias'),
    dict(Task="Semana 09 – 13 Junio", Start='2025-06-09', Finish='2025-06-15', Resource='Cruce BBDD y Cartolas'),
    dict(Task="Semana 16 – 20 Junio", Start='2025-06-16', Finish='2025-06-22', Resource='Conciliación de cartolas bancarias con GPI y Dashboard de Racional'),
    dict(Task="Semana 23 – 26 Junio", Start='2025-06-23', Finish='2025-06-29', Resource='Incorporación del resto de las Fintech al proceso de conciliación'),
    dict(Task="Semana 30 – 04 Julio", Start='2025-06-30', Finish='2025-07-06', Resource='Continuación del proceso de incorporación de FinTech'),
    dict(Task="Semana 07 – 11 Julio", Start='2025-07-07', Finish='2025-07-13', Resource='Análisis de casos críticos detectados en los resultados de las conciliaciones'),
    dict(Task="Semana 14 – 18 Julio", Start='2025-07-14', Finish='2025-07-20', Resource='Entrega de reportes. Integración de código Python con visualización en Power BI'),
    dict(Task="Semana 21 – 25 Julio", Start='2025-07-21', Finish='2025-07-27', Resource='Integración con Power BI. Entrega de reportes. Toma de decisiones en base a resultados')
]

colors = [
    'rgb(0, 102, 204)', 'rgb(0, 153, 255)', 'rgb(0, 204, 204)', 'rgb(102, 204, 102)',
    'rgb(255, 204, 0)', 'rgb(255, 153, 51)', 'rgb(255, 102, 102)', 'rgb(204, 0, 102)'
]

resources = [item['Resource'] for item in df]
color_map = dict(zip(resources, colors))

fig = ff.create_gantt(df, colors=color_map, index_col='Resource', show_colorbar=True, group_tasks=True)

# Reordenar manualmente los traces para que la leyenda siga el orden de 'resources'
ordered_traces = []
for resource in resources:
    for trace in fig.data:
        if trace.name == resource:
            ordered_traces.append(trace)
fig.data = tuple(ordered_traces)

fig.update_layout(
    width=2000,  # Aumenta el ancho
    height=700,  # Aumenta el alto
    font=dict(size=17),
    legend=dict(
        font=dict(size=18),  # Achica la leyenda
        title_font=dict(size=16),
        traceorder="normal",
        itemsizing='constant'
    )
)

fig.show()